# 180328
view, purchase관련 DataFrame을 보는데 얘네를 JOIN시켜봄. 결과는 아래와 같다

### 1. len(view) = 2833180
### 2. len(purchase) = 168996
### 3. len(view JOIN purchase) = 276

궁극적으로 우리가 해야 하는 건 각 고객에게 추천 리스트를 제공하는 건데

이 때 고객(과 쿠폰을 연결시키는 연결고리로 view와 purchase를 사용할 수 있다

---------------------

그럼 각각의 DataFrame으로 고객과 쿠폰을 연결시킨다면 __어떤 의미__를 가지는가?

### 1. view = 본 애들만 기준으로 추천하겠다
> 실수로 본 쿠폰들도 추천할 수 있음. 데이터 양이 많음

> 고객이 한번이라도 들여다본 쿠폰을 추천하자면 얘로 사용

### 2. purchase = 산 애들만 기준으로 추천하겠다
> 꼭 view에 있지 않아도, 예를 들어 지인추천으로 홈페이지를 들여다보지 않고 바로 구매하는 경우

> 아래의 봤으면서 산 애들을 기준으로 추천하는 것보다 현실을 더 넓게 커버한다

### 3. vp = 봤으면서 산 애들만 기준으로 추천하겠다
> 이 기준은 커버하는 현실이 제일 좁고 데이터 수가 많지 않음

> 만약 이 회사입장에서 충성고객의 activity를 고려한다면 얘로 사용해봄직함

> 충성고객? 홈페이지에서 어떤 쿠폰이 새로 올라왔나 view하고 purchase하는 부류

---------------------

## 그럼 본 애들, 산 애들 따로는 추천 못하나?
### 모델을 각각 따로 만든 후 test한 값을 합친다?
ex) 
1. a = view_model(Xtest), b = purchase_model(X_test)
2. a와 b는 각 쿠폰에 대한 PURCHASE_FLAG (아마 pd.Series형태)
3. 얘네를 더함

# 모델링에 쓸 데이터 기준
1. view = 본 애들만 추천하겠다
2. purchase = 산 애들만 추천하겠다
3. view and purchase = 봤으면서 산 애들만 추천하겠다
4. view or purchase = 보거나 산 애들이면 추천하겠다

----------

## coupon area.csv
1. 쿠폰과 그 사용위치를 알려줌

2. 여러 위치에서 사용되는 쿠폰이 있어서 138185개

3. unique한 coupon_id는 19368개이며 pk는 세 컬럼 전부

4. 하지만 coupon_list는 unique한 coupon_id가 19413개

5. 그럼 __45개__만큼 모자란 area정보는 어디있는가? coupon_list에 있긴 있음

## prefecture locations.csv
1. 47개 prefecture에 대한 위도, 경도
2. row도 47개

# training coupon area VS coupon list
1. coupon_area는 쿠폰ID, 모든 가능한 위치정보
2. coupon_list의 쿠폰ID, 위치정보도 coupon_area와 같음
3. 그럼 왜 unique한 coupon_id가 차이가 날까?
# 4. coupon_area에 없는 애들은 coupon_list에서 빼고 해라?

-------------

coupon list와 coupon area의 area정보의 분포를 살펴보니

많이 다르지 않다 (coupon_area_eda 참조)
### coupon area를 list와 JOIN(45개 제외됨)
### coupon_list만 써서 training시킬지
### 둘 다 해보기

--------------

# JOIN결과

* purchase = 약 17만개 (168996 rows)
* view = 약 300만개 (2833180 rows)


#### 1. User <-> View <-> Coupon <-> Prefecture

= 2517206 rows × 41 columns

#### 2. User <-> View <-> Coupon <-> Area <-> Prefecture

= 2513829 rows × 42 columns

#### 3. User <-> Purchase <-> Coupon <-> Prefecture

= 168996 rows × 38 columns

#### 4. User <-> Purchase <-> Coupon <-> Area <-> Prefecture

= 168787 rows × 39 columns

#### 5. User <-> View X Purchase <-> Coupon <-> Prefecture

= ~~276 rows × 44 columns~~ (12만개)

#### 6. User <-> View X Purchase <-> Coupon <-> Area <-> Prefecture

= ~~276 rows × 45 columns~~ (12만개)

## 왜 coupon X area X prefecture는 갯수가 더 적을까?
= coupon_area에 의해 45개 쿠폰이 coupon_list에서 제거되는데 그 영향이 JOIN에 cascade되었을 것이다

----------------------------------------------------------------------------
----------------

# 180329

## user_list csv

1. 고객들은 30대~50대가 주로 많음
2. 남자 11890명, 여자 10983명, 총 22873명
3. Tokyo, Kanagawa, Osaka에 고객많음
4. 탈퇴날짜는 922명만 기록되있음
==> 922명만 탈퇴했다
5. 고객들 등록율은 500~1000으로 고만고만한 편.
다만 __2010년 11월, 2011년 5월__에 팍 치솟음
6. 2011년 6월 ~ 2012년 7월 사이에 탈퇴율이 증가하다 감소함.
2012년 2월에 탈퇴율이 확 줄음

# 쿠폰이던 고객이던 Tokyo, Kanagawa, Osaka가 많음

--------------

# 180330
그럼 각각의 DataFrame으로 고객과 쿠폰을 연결시킨다면 __어떤 의미__를 가지는가?

### 1. view = 본 애들만 기준으로 추천하겠다
> 실수로 본 쿠폰들도 추천할 수 있음. 데이터 양이 많음

> 고객이 한번이라도 들여다본 쿠폰을 추천하자면 얘로 사용

### 2. purchase = 산 애들만 기준으로 추천하겠다
> 꼭 view에 있지 않아도, 예를 들어 지인추천으로 홈페이지를 들여다보지 않고 바로 구매하는 경우

> 아래의 봤으면서 산 애들을 기준으로 추천하는 것보다 현실을 더 넓게 커버한다

### 3. view 중 PURCHASE_FLG==1  = 봤으면서 산 애들만 기준으로 추천하겠다
> 이 기준은 커버하는 현실이 제일 좁고 데이터 수가 많지 않음

> 만약 이 회사입장에서 충성고객의 activity를 고려한다면 얘로 사용해봄직함

> 충성고객? 홈페이지에서 어떤 쿠폰이 새로 올라왔나 view하고 purchase하는 부류

> # 본 것 중 산 애들은 4%이다 (280만개 중 13만개)

# 모델링에 쓸 데이터 기준
1. view = 본 애들만 추천하겠다
2. purchase = 산 애들만 추천하겠다
3. view 중 PURCHASE_FLAG==1 = 봤으면서 산 애들만 추천하겠다
4. view or purchase = 보거나 산 애들이면 추천하겠다